In [29]:
import os
import pandas as pd
import re

In [2]:
r1_hs_seqs_path = '/Users/stephaniecrilly/Library/CloudStorage/Box-Box/kortemmelab/home/scrilly/helix_sliding/20231212_final_analysis_r1_order/20231221_final_order/20231221_all_design_sets_r1_order.csv'
r2_ssd_seqs_path = '/Users/stephaniecrilly/Library/CloudStorage/Box-Box/kortemmelab/home/scrilly/helix_sliding/20250604_r2_hs_lib/metric_files/ssd/ssd_designs_passing.csv'
r2_msd_seqs_path = '/Users/stephaniecrilly/Library/CloudStorage/Box-Box/kortemmelab/home/scrilly/helix_sliding/20250604_r2_hs_lib/metric_files/msd/msd_designs_passing.csv'
r2_msd_exp_bbs_seqs_path = '/Users/stephaniecrilly/Library/CloudStorage/Box-Box/kortemmelab/home/scrilly/helix_sliding/20250604_r2_hs_lib/metric_files/msd_exp_bbs/exp_bbs_msd_designs_passing.csv'
r2_msd_scale_up_seqs_path = ''

outdir = '/Users/stephaniecrilly/Library/CloudStorage/Box-Box/kortemmelab/home/scrilly/helix_sliding/20250604_r2_hs_lib/final_order'

In [7]:
#make df with additional control seqs from Bram and 375 and point mutants
bespoke_ctrls_dict = {'bmp38' : 'GARLAQLKQERAALKQRLAALDQEIAALEWQIQSDPRKKQLYQRLLALISDRFALEQRIAALDQEIAALEAG', 
                 'bmp53' : 'GARLEELIAERLRLVGDLVDLDREIAALEQQIQSDPRKKQLEQRLAALKQERAALEQRIAALDWEIADLDDG', 
                 'bmp31' : 'GARLAQLKQERAALKQRLAALDQEIAALEWQIQSDPRKKQLEQRLAALKQERAALEQRIAALDQEIAALEAG', 
                 'bmp36' : 'GARLAQLKQERAALEQRLAALDQEIAALEWQIQSDPRKKQLEQKLLELQAERLRLIGDIVNLDQQILNLEAG', 
                 'bmp42' : 'GARLAQLKQERAALKQRLDALDQEIAALEWQIQSDPRKKQLYQRLLELIGERLALMGDIFELDVEIAALEAG', 
                 'bmp45' : 'GARLAQLKQERAALKQRLEALDQEIAALEWQIQSDPRKKQLLQRLYELFGERLALFGDIFDLDVEIAALEAG', 
                 'bmp51' : 'GARLAQLKQERAALKQRLDALEQEIAALEWQIQSDPRKKQLQQRLRQLYGERLRLESDIFDLDVEIAALEAG', 
                 'bmp57' : 'GARLAQLKQERAALKQRLEALDQEIAALEWQIQSDPRKKQLLQRLRRLYGERLALFGDIFDLDVEIAALEAG', 
                 'bmp23' : 'GARLAQLKQERAALKQRLAALDQEIAALEWQIQSPDRKKQLEQRLAALKQERAALEQRIAALDQEIAALEAG',
                 '375' : 'EELKRIEEEIAAIEREIARAEEKLKAQESDPRKKGLQAEREKLLEKLAELRKERERLSRLEEELRRRLTELRRRLE',
                 '375-R74W' : 'EELKRIEEEIAAIEREIARAEEKLKAQESDPRKKGLQAEREKLLEKLAELRKERERLSRLEEELRRRLTELRRWLE',
                 '375-L75W' : 'EELKRIEEEIAAIEREIARAEEKLKAQESDPRKKGLQAEREKLLEKLAELRKERERLSRLEEELRRRLTELRRRWE',
                 '375-77W' : 'EELKRIEEEIAAIEREIARAEEKLKAQESDPRKKGLQAEREKLLEKLAELRKERERLSRLEEELRRRLTELRRRLEW',
                 '375-QatF' : 'QELKRIEQEIAAIEQEIARAEQKLKAQESDPRKKGLQAEREKLLEKLAELRKERERLSRLEEELRRRLTELRRRLEW',
                 '375-QatF-cdel' : 'QELKRIEQEIAAIEQEIARAEQKLKAQESDPRKKGLQAEREKLLEKLAELRKERERLSRLEEELRRRLTE',
                 '375-ad-to-A' : 'EEAKRAEEEAAAAEREAARAEEKAKAQESDPRKKGLQAEAEKALEKAAEARKEAERASRLEEELRRRLTELRRRLE',
                 '375-ad-to-A-cdel' : 'EEAKRAEEEAAAAEREAARAEEKAKAQESDPRKKGLQAEAEKALEKAAEARKEAERASRLEEELRRRLTE', 
                 '375-cdel' : 'EELKRIEEEIAAIEREIARAEEKLKAQESDPRKKGLQAEREKLLEKLAELRKERERLSRLEEELRRRLTE', 
                 '375-G53Q' : 'EELKRIEEEIAAIEREIARAEEKLKAQESDPRKKQLQAEREKLLEKLAELRKERERLSRLEEELRRRLTELRRRLE', 
                 '375-G35Q-cdel' : 'EELKRIEEEIAAIEREIARAEEKLKAQESDPRKKQLQAEREKLLEKLAELRKERERLSRLEEELRRRLTE', 
                 '375-E14Q' : 'EELKRIEEEIAAIQREIARAEEKLKAQESDPRKKGLQAEREKLLEKLAELRKERERLSRLEEELRRRLTELRRRLE', 
                 '375-E14Q-cdel' : 'EELKRIEEEIAAIQREIARAEEKLKAQESDPRKKGLQAEREKLLEKLAELRKERERLSRLEEELRRRLTE', 
                 '375-E14Q-G35Q' : 'EELKRIEEEIAAIQREIARAEEKLKAQESDPRKKQLQAEREKLLEKLAELRKERERLSRLEEELRRRLTELRRRLE',
                 '375-E14Q-G35Q-cdel' : 'EELKRIEEEIAAIQREIARAEEKLKAQESDPRKKQLQAEREKLLEKLAELRKERERLSRLEEELRRRLTE',
                 '375-G35A' : 'EELKRIEEEIAAIEREIARAEEKLKAQESDPRKKALQAEREKLLEKLAELRKERERLSRLEEELRRRLTELRRRLE', 
                 '375-G35A-cdel' : 'EELKRIEEEIAAIEREIARAEEKLKAQESDPRKKALQAEREKLLEKLAELRKERERLSRLEEELRRRLTE', 
                 '375-L39P' : 'EELKRIEEEIAAIEREIARAEEKLKAQESDPRKKGPQAEREKLLEKLAELRKERERLSRLEEELRRRLTELRRRLE',
                 '375-L39P-cdel' : 'EELKRIEEEIAAIEREIARAEEKLKAQESDPRKKGPQAEREKLLEKLAELRKERERLSRLEEELRRRLTE',
                 '375-L36K' : 'EELKRIEEEIAAIEREIARAEEKLKAQESDPRKKGKQAEREKLLEKLAELRKERERLSRLEEELRRRLTELRRRLE',
                 '375-L36K-cdel' : 'EELKRIEEEIAAIEREIARAEEKLKAQESDPRKKGKQAEREKLLEKLAELRKERERLSRLEEELRRRLTE',
                 '375-L36Q' : 'EELKRIEEEIAAIEREIARAEEKLKAQESDPRKKGQQAEREKLLEKLAELRKERERLSRLEEELRRRLTELRRRLE',
                 '375-L36Q-cdel' : 'EELKRIEEEIAAIEREIARAEEKLKAQESDPRKKGQQAEREKLLEKLAELRKERERLSRLEEELRRRLTE',
                 '375-ALFA-LtoA' : 'EELKRIEEEIAAIEREIARAEEKLKAQESDPRKKGLQAEREKLLEKLAELRKERERLSRAEEEARRRATELRRRLE',
                 '375-ALFA-LtoA-cdel' : 'EELKRIEEEIAAIEREIARAEEKLKAQESDPRKKGLQAEREKLLEKLAELRKERERLSRAEEEARRRATE'}

#make df with columns 'Name and 'sequence' from dict
bespoke_ctrls_df = pd.DataFrame.from_dict(bespoke_ctrls_dict, orient='index', columns=['sequence']).reset_index().rename(columns={'index': 'Name'})
bespoke_ctrls_df.shape

(34, 2)

In [16]:
#take MSD sequences that are low RMSD to only one state, refilter for plddt > 90 and see how many
r2_msd_exp_bbs_min0_path = '/Users/stephaniecrilly/Library/CloudStorage/Box-Box/kortemmelab/home/scrilly/helix_sliding/20250604_r2_hs_lib/metric_files/msd_exp_bbs/exp_bbs_msd_designs_passing_min0_only.csv'
r2_msd_exp_bbs_min2_path = '/Users/stephaniecrilly/Library/CloudStorage/Box-Box/kortemmelab/home/scrilly/helix_sliding/20250604_r2_hs_lib/metric_files/msd_exp_bbs/exp_bbs_msd_designs_passing_min2_only.csv'
r2_msd_min0_path = '/Users/stephaniecrilly/Library/CloudStorage/Box-Box/kortemmelab/home/scrilly/helix_sliding/20250604_r2_hs_lib/metric_files/msd/msd_designs_passing_min0_only.csv'
r2_msd_min2_path = '/Users/stephaniecrilly/Library/CloudStorage/Box-Box/kortemmelab/home/scrilly/helix_sliding/20250604_r2_hs_lib/metric_files/msd/msd_designs_passing_min2_only.csv'

r2_msd_exp_bbs_min0_df = pd.read_csv(r2_msd_exp_bbs_min0_path)
print(r2_msd_exp_bbs_min0_df.shape)
r2_msd_exp_bbs_min0_df = r2_msd_exp_bbs_min0_df.query('avg_plddt_no_loop > 90 & min0_all_rmsd_no_loop < 1.0  & avg_pae_no_loop < 3', engine='python').copy()
print(r2_msd_exp_bbs_min0_df.shape)

r2_msd_exp_bbs_min2_df = pd.read_csv(r2_msd_exp_bbs_min2_path)
print(r2_msd_exp_bbs_min2_df.shape)
r2_msd_exp_bbs_min2_df = r2_msd_exp_bbs_min2_df.query('avg_plddt_no_loop > 90  & min2_all_rmsd_no_loop < 1.0  & avg_pae_no_loop < 3', engine='python').copy()
print(r2_msd_exp_bbs_min2_df.shape)

r2_msd_min0_df = pd.read_csv(r2_msd_min0_path)
print(r2_msd_min0_df.shape)
r2_msd_min0_df = r2_msd_min0_df.query('avg_plddt_no_loop > 90  & min0_all_rmsd_no_loop < 1.0 & avg_pae_no_loop < 3', engine='python').copy()
print(r2_msd_min0_df.shape)

r2_msd_min2_df = pd.read_csv(r2_msd_min2_path)
print(r2_msd_min2_df.shape)
r2_msd_min2_df = r2_msd_min2_df.query('avg_plddt_no_loop > 90  & min2_all_rmsd_no_loop < 1.0  & avg_pae_no_loop < 3', engine='python').copy()
print(r2_msd_min2_df.shape)


(3611, 45)
(1297, 45)
(902, 45)
(258, 45)
(10882, 40)
(3782, 40)
(670, 40)
(114, 40)


#500 patterned scramble
#500 full scramble
#500 a and d to Ala 
#split 250/250 MPNN and F2s

In [32]:
def heptad_to_upper(input_string, split_char = 'r '):
    split_1 = input_string.split(split_char)[0]
    split_2 = input_string.split(split_char)[1]
    split_2 = split_2.upper()
    
    new_string = split_1+split_char+split_2

    return new_string

# def make_heptad_strings(reg_string, seq_string):
#     heptads_dict = {'a_residues' : [],
#                     'b_residues' : [],
#                     'c_residues' : [],
#                     'd_residues' : [],
#                     'e_residues' : [],
#                     'f_residues' : [],
#                     'g_residues' : []}
    
#     for index, char in enumerate(reg_string):
#         if char.isupper():
#             key = char.lower() + '_residues'
#             if key in heptads_dict:
#                 heptads_dict[key].append(seq_string[index])
    
#     for k, v in heptads_dict.items():
#         heptads_dict[k] = ''.join(v)

#     return heptads_dict

####


def heptad_a_d_to_ala(reg_string, seq_string):
    ala_dict = {}

    og_seq = re.split(r'\s+', seq_string)[1]
    
    for index, char in enumerate(reg_string):
        if char.isupper():
            if char in ['A', 'D']:
                seq_string = seq_string[:index] + 'A' + seq_string[index+1:]
            else:
                continue
    
    new_seq = re.split(r'\s+', seq_string)[1]

    ala_dict[og_seq] = new_seq

    return ala_dict


def get_heptad_res_identities(input_df):

    h1_dfs_to_concat = []
    h2_dfs_to_concat = []

    for index, row in input_df.iterrows():
        h1_reg = row['h1_reg']
        new_h1_reg = heptad_to_upper(h1_reg)

        h1_seq = row['h1_seq']

        h1_dict = make_heptad_strings(new_h1_reg, h1_seq)
        h1_df = pd.DataFrame([h1_dict])
        h1_df['design_id'] = row['design_id']

        h1_dfs_to_concat.append(h1_df)

        h2_reg = row['h2_reg']
        new_h2_reg = heptad_to_upper(h2_reg)

        h2_seq = row['h2_seq']

        h2_dict = make_heptad_strings(new_h2_reg, h2_seq)
        h2_df = pd.DataFrame([h2_dict])
        h2_df['design_id'] = row['design_id']

        h2_dfs_to_concat.append(h2_df)
    
    h1_dfs = pd.concat(h1_dfs_to_concat)
    h2_dfs = pd.concat(h2_dfs_to_concat)

    h1_h2_df = h1_dfs.merge(h2_dfs, on='design_id', suffixes=['_h1', '_h2'])

    new_df = input_df.merge(h1_h2_df, on='design_id')

    return new_df


In [ ]:
test_heptad = 'register       abcdefgabcdefgabcdefgabcd          '
test_str = 'sequence PLREKLLELKERLKELLELKLEELSRLEEELRRRLTEEERR'

import re
new_heptad = heptad_to_upper(test_heptad)
print(new_heptad)


test_dict = heptad_a_d_to_ala(new_heptad, test_str)
print(test_dict)

#need to exclude alfa tag

register       ABCDEFGABCDEFGABCDEFGABCD          
{'PLREKLLELKERLKELLELKLEELSRLEEELRRRLTEEERR': 'PLREKLAELAERLAELAELKAEEASRLAEEARRRLTEEERR'}


In [ ]:
#generate post alfa truncated seqs from r1_hs_seqs and all others
#then remove any of duplciates already in r2_ssd (should be 64)

#update socket utils with new functions and push to cleaning
#test for making a and d to Ala